In [1]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pymysql
import re
from urllib.parse import urlparse, parse_qs

# mysql db connect
db = pymysql.connect(host='52.78.23.232', user='root', password='readers7', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

In [2]:
# ------ 연재 웹툰 크롤링
# 요일 탭 클릭
def setDay(dayvalue):
    day = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div[3]/div[2]/ul/li[" + str(dayvalue) + "]")
    day.click()
    browser.implicitly_wait(3)

def day_webtoon(day, num):
    webtoon = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div[3]/div[3]/ul/li[" + str(num) + "]/a")
    
    # 작품 불러오기
    toon_url = webtoon.get_attribute('href')
    
    # ID
    toon_id = 'kakao_' + str(toon_url).replace("https://page.kakao.com/home?seriesId=", '')
    
    # 요일 추출
    if day == 1: weekday = 'mon'
    if day == 2: weekday = 'tue'
    if day == 3: weekday = 'wed'
    if day == 4: weekday = 'thu'
    if day == 5: weekday = 'fri'
    if day == 6: weekday = 'sat'
    if day == 7: weekday = 'sun'
    cursor1.execute(weekday_sql, (toon_id, weekday))
    
    # 제목
    name = webtoon.find_element_by_class_name('jsx-3847971416')
    toon_name = name.text
    
    # 작가
    writer = webtoon.find_element_by_class_name('author')
    wrt_name = writer.text
    
    # 설명
    description = webtoon.find_element_by_class_name('captionWrap')
    desc = description.find_element_by_class_name('jsx-1134470472')
    toon_desc = desc.text
    
    # 썸네일 주소
    webtoon = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div[3]/div[3]/ul/li[" + str(num) + "]/a/div[1]/div[1]/div/img")
    browser.execute_script("arguments[0].scrollIntoView(true);", webtoon) # Scroll to this webtoon element
                         # Scroll to bottom is "window.scrollTo(0,document.body.scrollHeight)"
    toon_thumb_url = 'https://page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a'
        # 이미지 로딩 안 된 현상 해결
    while toon_thumb_url == 'https://page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a':
        toon_thumb_url = webtoon.get_attribute('src')
        
    # toon_info
    cursor1.execute(sel_sql)
    result = cursor1.fetchall()
    if (toon_id not in result):
        cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'kakao', wrt_name, 'X', toon_thumb_url))
        #cursor3.execute(genre_sql, (toon_id, genre)) # 장르 정보 없음
    db.commit()

In [3]:
sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"
'''
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
'''


browser = webdriver.Chrome('C:/Users/Administrator/OneDrive/Readers/Crawling/chromedriver.exe')
browser.get('https://page.kakao.com/main?categoryUid=10&subCategoryUid=1000')

for day in range(1, 8):
    # Scroll to Top 안 해주면 element 분석 못함;
    # 크롤링 하는 중에 페이지 내에서 내가 스크롤 해도 갑자기 오류뜸ㅋ 예민한놈
    browser.execute_script("window.scrollTo(0,0);")
    
    setDay(day)
    print('*********************')
    print(day, 'day')
    for num in range(1, 100):
        try:
            day_webtoon(day, num)
        except:
            break;

*********************
1 day
-- 1
학사재생
-- 2
후궁계약
-- 3
찬란한 액션 유치원
-- 4
과격자매단
-- 5
트레이스-라스트
-- 6
웽툰
-- 7
소금물 방
-- 8
창공의 크로이스
-- 9
정상회담 : 스틸레인3
-- 10
안티 안티 엔젤
-- 11
막내
-- 12
해동고 짱짱맨 시즌2
-- 13
강철비
-- 14
*********************
2 day
-- 1
달빛조각사
-- 2
아비무쌍
-- 3
우리 집 우렁이는
-- 4
묘한 인생
-- 5
웽툰


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53764524' for key 'PRIMARY'")
  self._do_get_result()


-- 6
엄마는 외국인 시즌3,4 [시즌4 연재]
-- 7
겟 레디 위드 미
-- 8
빌린 몸
-- 9
해골병사는 던전을 지키지 못했다
-- 10
미러게임
-- 11
공중정원
-- 12
헬로 마이 안드로이드
-- 13
지옥사원
-- 14
체이서
-- 15
미생 두번째 시즌
-- 16
저작재잘
-- 17
*********************
3 day
-- 1
여혜
-- 2
FFF급 관심용사
-- 3
소녀샵
-- 4
웽툰
-- 5
왕자님의 막대사탕
-- 6
키스해주세요
-- 7
헤븐즈 소울
-- 8
포갓레인저 시즌3
-- 9
카운터
-- 10
좋아하면 울리는 시즌7
-- 11
마애 (磨崖)
-- 12
파티시에
-- 13
아임 펫! 시즌4
-- 14
*********************
4 day
-- 1
그래서 오늘도 삽니다
-- 2
샬롯에게는 다섯 명의 제자가 있다
-- 3
전지적 짝사랑 시점
-- 4
블랙윈터
-- 5
꽃을 문 짐승
-- 6
승은이 망극하옵니다
-- 7
웽툰
-- 8
엄마는 외국인 시즌3,4 [시즌4 연재]
-- 9


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_48225112' for key 'PRIMARY'")
  self._do_get_result()


스트레서
-- 10
배고픈 자들의 도시
-- 11
머리 위로 부는 바람
-- 12
크라이시스
-- 13
사귀자
-- 14
구리의 구리구리 컴백
-- 15
요괴난전
-- 16
*********************
5 day
-- 1
랑데부
-- 2
뮈엘라의 수사관
-- 3
유리의 벽
-- 4
여혜
-- 5


...

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53956161' for key 'PRIMARY'")
  self._do_get_result()


-- 4
부기영화
-- 5
전학생 오덕구
-- 6
순정 히포크라테스
-- 7
표류감옥
-- 8
애니멀 아이즈
-- 9
Go Bananas
-- 10
*********************
7 day
-- 1
대마법사의 딸
-- 2
내 일진이 이렇게 귀여울리가 없어
-- 3
눈부시게 빛나는
-- 4
대디보이
-- 5
하필 너야?!
-- 6
일진사냥
-- 7
하렘생존기
-- 8
항변신
-- 9
스타폴:별을 위한 노래
-- 10
좋아하면 울리는 시즌7
-- 11
Go Bananas

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_48211158' for key 'PRIMARY'")
  self._do_get_result()



-- 12


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'kakao_53491650' for key 'PRIMARY'")
  self._do_get_result()


In [4]:
# ------ 회차 정보
browser = webdriver.Chrome('C:/Users/Administrator/OneDrive/Readers/Crawling/chromedriver.exe')

sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "kakao"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url'] + '&page='
    page = 1
    #print('*********************************************')
    #print('<' + toon_id + '>')
    
    num = 1 # episode 개수 탐색
    while num < 99999:
        try:
            browser.get(toon_url+ str(page))
            episode = browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div/div/div[3]/ul/li[" + str(num) + "]")
            browser.execute_script("arguments[0].scrollIntoView(true);", episode) # Scroll to this episode
            browser.implicitly_wait(3)
            
            # 에피소드 제목
            name = episode.find_element_by_xpath('div[2]/dl/dt/p')
            epi_name = name.text
            
            
            # 여기다 cursor2.fetchall 넣으면 될듯....
            
            # 에피소드 링크
            epi_id = episode.get_attribute('data-productid')
            epi_url = "https://page.kakao.com/viewer?productId=" + epi_id
            
            # 썸네일 링크
            thumb = episode.find_element_by_xpath("div[1]/div/div/img")
                # 이미지 로딩 안 된 현상 해결
            epi_thumb_url = 'https://page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a'
            while epi_thumb_url == 'https://page.kakao.com/static/common/bg_noImg.png?d0b47a2245ca6f22b0660775a3d5c27a':
                epi_thumb_url = thumb.get_attribute('src')
            
            # 업데이트 날짜 추출
            date = episode.find_element_by_xpath("div[2]/dl/dd")
            epi_date = date.text.replace(".", "-")
            
            
            
            cursor2.execute(sel_dup_sql, (toon_id, epi_name))
            if cursor2.fetchall():
                num += 1
                continue
            else:
                cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                db.commit()
                
            num += 1
        except:
            # 다음 페이지가 유효한 지 검사
            browser.get(toon_url + str(page+1))
            num = 1
            try:
                browser.find_element_by_xpath("//*[@id=\"root\"]/div[3]/div/div/div[3]/ul/li[" + str(num) + "]")
                page += 1
                continue
            except NoSuchElementException:
                break